# Урок 5. Проверка статистических гипотез. P-значения. Доверительные интервалы. A/B-тестирование

In [1]:
import numpy as np
from math import sqrt
from scipy import stats

In [2]:
# доверительный интервал
def confidence_interval(mean, norm_value, std, n):
    return mean - norm_value * std/sqrt(n), mean + norm_value * std/sqrt(n)

**Задача 1**

Известно, что генеральная совокупность распределена нормально с известным средним квадратическим отклонением 16. Найти доверительный интервал для оценки математического ожидания mu с надёжностью 0.95, если выборочное среднее равно 80, а объём равен 256.

In [3]:
std = 16 # среднее квадратическое отклонение
p = 0.95 # доверительная вероятность.
M = 80 #выборочное среднее
n = 256 #размер выборка

In [4]:
alpha = 1 - p
z = round(stats.norm.ppf(1 - alpha / 2),2)
z

1.96

In [5]:
# доверительный интервал
confidence_interval(M, z, std, n)

(78.04, 81.96)

**Задача 2**

Продавец утверждает, что средний вес пачки печенья составляет 200 г. Из партии извлечена выборка из 10 пачек. Вес каждой пачки составляет:

202, 203, 199, 197, 195, 201, 200, 204, 194, 190
Известно, что их веса распределены нормально.

Верно ли утверждение продавца, если учитывать, что уровень значимости равен 1%?
Найдите P-значение для данного теста.

In [6]:
cookies = np.array([202, 203, 199, 197, 195, 201, 200, 204, 194, 190])

m = 200 #gr
n = 10
alpha = 0.01

In [7]:
t1 = stats.t.ppf(alpha / 2, df=n-1)
t2 = stats.t.ppf(1 - alpha /2, df=n-1)

t1, t2

(-3.24983554401537, 3.2498355440153697)

In [8]:
def statistic(cookies: np.ndarray, m):    
    sigma = cookies.std(ddof=1)
    n = cookies.shape[0]
    
    return (cookies.mean() - m)/sigma/n**(1/2)

In [9]:
S = statistic(cookies, m)
S

-0.10651074037450894

In [10]:
t1<=S<=t2 # S не попало в критические область - утверждение продавца верно

True

Найдите P-значение для данного теста.

In [11]:
p_left = stats.t.cdf(S, df=n - 1)
p_right = 1 - stats.t.cdf(S, df=n - 1)

pvalue = round(2 * min(p_left, p_right), 2)

pvalue

0.92

**Задача 3**

Дана выборка диаметров подшипников из примера 1 с занятия 5:

samples = [0.6603, 0.9466, 0.5968, 1.3792, 1.5481, 0.7515, 1.0681, 1.1134,
           1.2088, 1.701 , 1.0282, 1.3579, 1.0191, 1.1784, 1.1168, 1.1372,
           0.7273, 1.3958, 0.8665, 1.5112, 1.161 , 1.0232, 1.0865, 1.02  ]
Предполагая, что диаметры подшипников распределены нормально, проверьте гипотезу о том, что дисперсия случайной величины равна 0.0625 при уровне значимости alpha = 0.05. Что для этого нужно знать:

Альтернативная гипотеза двухсторонняя.
Статистика для теста: H = (n - 1) * sample_variance / variance, где n - число элементов в выборке, sample_variance - несмещённая оценка дисперсии, variance - утверждаемая нулевой гипотезой дисперсия.
Эта статистика в предположении верности нулевой гипотезы имеет распределение хи-квадрат с параметром df = n - 1. Её квантили можно найти с помощью функции scipy.stats.chi2.ppf.

In [12]:
samples = np.array( [0.6603, 0.9466, 0.5968, 1.3792, 1.5481, 0.7515, 1.0681, 1.1134, 1.2088, 1.701 , 1.0282, 1.3579, 1.0191, 1.1784, 1.1168, 1.1372, 0.7273, 1.3958, 0.8665, 1.5112, 1.161 , 1.0232, 1.0865, 1.02 ])
Dx0 = 0.0625
alpha = 0.05

In [13]:
#Альтернативная гипотеза двухсторонняя
def H(samples: np.ndarray, Dx0):
    
    n = samples.shape[0]
    sample_variance = samples.var(ddof=1)
    variance = Dx0
    
    return (n - 1) * sample_variance / variance

In [14]:
def confidence_interval2(samples: np.ndarray, alpha):
    
    n = samples.shape[0]
    sample_variance = samples.var(ddof=1)
    
    t1 = stats.chi2.ppf(alpha / 2, df=n-1)
    t2 = stats.chi2.ppf(1 - alpha / 2, df=n-1)
    
    return (n-1)*sample_variance / t1, (n-1)*sample_variance / t2

In [15]:
S = H(samples=samples, Dx0=Dx0)
S

28.720541753333343

In [16]:
t1 = stats.chi2.ppf(alpha / 2, df=n-1)
t2 = stats.chi2.ppf(1 - alpha / 2, df=n-1)
t1, t2

(2.7003894999803584, 19.02276779864163)

In [17]:
t1 <= S <= t2

False

In [18]:
ci = confidence_interval2(samples=samples, alpha=alpha)
ci

(0.15357196267702494, 0.047143907775453915)

In [19]:
ci[1] <= Dx0 <= ci[0]

True

**Задача 4**

Утверждается, что шарики для подшипников, изготовленные автоматическим станком, имеют средний диаметр 17 мм. Используя уровень значимости 0.025 и правостороннюю критическую область, проверить данную гипотезу при условии, что в выборке из 100 шариков средний диаметр оказался равным 17.5 мм, а дисперсия известна и равна 4 мм.

In [20]:
mean = 17.5
alpha = 0.025

# Правосторонняя критическая область: Ωα=(t1−α,∞).
t = stats.norm.ppf(1 - alpha)
t

1.959963984540054

In [21]:
T = 5*(mean - 17)
T

2.5


Значение статистики T попало в критическую область, поэтому гипотезу, что шарики для подшипников, изготовленные автоматическим станком, имеют средний диаметр 17 мм, отвергаем